In [ ]:
from IPython.display import YouTubeVideo
# Required imports
import json
import re
import zipfile as zp
import pandas as pd
import numpy as np
import emoji
import random
import pygal
import user_agents
#flask example (example.zip musi być w katalogu głównym)
import sys
sys.path.append('..')
from preprocess import preprocess, _aux_get_structure

In [ ]:
zip_path = "/home/jacek/Pobrane/Okon_Slowacki_30_11_2020.zip"
# zip_path = "/home/jacek/Pobrane/facebook-jacekstasiak75 222.zip"
# zip_path = "/home/jacek/Pobrane/facebook-jacekstasiak75.zip"
# zip_path = "/home/jacek/Pobrane/win10 unpacked/facebook-okoń 28-12-20.zip"
# zip_path = "/home/jacek/Pobrane/facebook-janpodkowa98.zip"

data = None
acc_activity = None
with zp.ZipFile(zip_path) as zip:
#     data = gen_pandas_table(zip)
    data = preprocess(zip)


In [ ]:
# wczytywanie aktywności poza facebookiem

with zp.ZipFile(zip_path) as zip:
    folders = get_structure(zip)
    
    json_fname = None
    for name in folders['ads_and_businesses']['__files']:
#         print(name[0])
        if name[0] == 'your_off-facebook_activity.json':
            json_fname = name[1]
            break
    
    if json_fname:
#         print('json fname')
        names = []
        types = []
        times = []
        with zip.open(json_fname) as f:
            jdata = json.loads(f.read())
            for act in jdata['off_facebook_activity']:
#                 print(act['name'])
                name = act['name']
                for i in act['events']:
                    names.append(name)
                    types.append(i['type'])
                    times.append(i['timestamp'])
            table = pd.DataFrame({'name': names, 'type': types, 'time': times})
            table.time = pd.to_datetime(table.time, unit='s')
            data['off_activity'] = table
        

In [ ]:
activ = data['off_activity']
display = None
arrs = [[], [], [], []]
for name, info in activ.groupby(activ.name):
    arrs[0].append(name)
    arrs[1].append(info.name.count())
    arrs[2].append(info.time.min())
    arrs[3].append(info.time.max())
#     print(info.name.count(), name, info.time.min(), info.time.max())
display = pd.DataFrame({'name': arrs[0], 'number of visits': arrs[1], 'first visited': arrs[2], 'last visit': arrs[3]})
display.sort_values('number of visits', ascending=False)[:10]
# display.sort_values('last visit', ascending=False)[:10]

In [ ]:
test = ['some', 'data']
test2 = [None, [1, 2]]
df = pd.DataFrame({'test': test, 'test2': test2})
df

In [ ]:
reac = data['reactions']
your_reac = reac[reac.giver == data['username']]
your_reac.groupby(your_reac.reaction).count().giver.sort_values(ascending=False)

your_recived_reac = reac[reac.reciver == data['username']]
your_recived_reac = your_recived_reac.groupby(your_recived_reac.reaction).count().giver.sort_values(ascending=False)

graph1 = pygal.Bar(showlegend=False)
graph1.add('', list(your_recived_reac))
graph1.x_labels = list(your_recived_reac.index)
graph1

graph1 = pygal.Bar(showlegend=False)
graph1.add('', list(your_recived_reac))
graph1.x_labels = list(your_recived_reac.index)
graph1

# reac

In [ ]:
# Funkcja tworząca wykres skumulowany, wypełniając brakujące dni zerami.

def get_cum_graph(time_events):
    a = time_events.groupby([time_events.dt.year, time_events.dt.dayofyear])
    t = pd.Series(pd.date_range(time_events.dt.date.min(), time_events.dt.date.max()))
    t = pd.date_range(time_events.min(), time_events.max())
    t = pd.Series(t.date)
    days_not_present = t.isin(time_events.dt.date)
    days_not_present = ~days_not_present
    days_not_present
    # t[days_with_frens]
    temp = pd.DataFrame(
        {
            'time': time_events,
            'value': [1] * len(time_events)# + [-1] * len(days_not_present == False)
        }
    )

    temp = pd.concat([
        temp, 
        pd.DataFrame(
            {
                'time': t[days_not_present],
                'value': [-1] * len(t[days_not_present])
            }
        )    
    ])
    temp = temp.sort_values(['time']).reset_index(drop=True)
    temp

    timespan = temp.time.max() - temp.time.min()
    time_index = []

    temp.time = pd.to_datetime(temp.time)

    # grouped = temp.groupby(temp.time.dt.year)

    # Grupowanie przedziałów zliczania liczby znajomych, w zależności od tego jak stare jest konto
    if timespan.days > 360 * 10:
        distance = 'Y'
        grouped = temp.groupby([temp.time.dt.year, temp.time.dt.year])
        time_index = pd.date_range(temp.time.min().date(), temp.time.max(), freq='Y')
    elif timespan.days > 360 * 2:
        distance = 'QS'
        grouped = temp.groupby([temp.time.dt.year, temp.time.dt.quarter])
        time_index = pd.date_range(temp.time.min().date(), temp.time.max(), freq='QS')
    elif timespan.days > 40:
        distance = 'W'
        grouped = temp.groupby([temp.time.dt.year, temp.time.dt.isocalendar().week])
        time_index = pd.date_range(temp.time.min().date(), temp.time.max(), freq='W')
    elif timespan.days > 90:
        distance = 'M'
        grouped = temp.groupby([temp.time.dt.year, temp.time.dt.month])
        time_index = pd.date_range(temp.time.min().date(), temp.time.max(), freq='M')
    else:
        grouped = temp.groupby([temp.time.dt.year, temp.time.dt.day])
        time_index = pd.date_range(temp.time.min().date(), temp.time.max(), freq='D')

    # zliczanie elementów które nie są '-1'
    b = grouped['value'].agg(lambda x: len(x[x != -1]))

    # cum sumowanie
    b = b.cumsum()

    return b, time_index


    

In [ ]:
# Wykresy z liczbą postów

post_data = None
with zp.ZipFile(zip_path) as zip:
    _folders = _aux_get_structure(zip)
    with zip.open(_folders['posts']['__files'][0][1]) as f:
        jdata = json.loads(f.read())
        tdata = {'time': [], 'title': []}
        
        if isinstance(jdata, dict):
            jdata = [jdata]
            
        for i in jdata:
            tdata['time'].append(i['timestamp'])
            tdata['title'].append(i.get('title'))
        post_data = pd.DataFrame(tdata)
        post_data.time = pd.to_datetime(post_data.time, unit='s')
        
post_data # Dodaj do data
post_data = pd.concat([post_data, pd.DataFrame({'time': [pd.Timestamp("2020-11-25 12:00:35")], 'title': ['Fake post.']})])

# # cum graph (zakres się dostosowywuje)
# vals, index = get_cum_graph(post_data['time'])
# char = pygal.Line(fill=True, x_label_rotation=-45, show_legend=False)
# char.x_labels = index.date
# char.add('', list(vals))

# # graph by years
# posts_by_year = post_data.groupby(post_data.time.dt.year).count()
# year_values = list(posts_by_year.time)
# year_indexes = list(posts_by_year.index)

# if len(posts_by_year) == 1:
#     year_values = [0] + year_values
#     year_indexes = [year_indexes[0] - 1] + year_indexes

# chart = pygal.Line(show_legend=False)
# chart.x_labels = year_indexes
# chart.add('', year_values)

# # graph by hours
gr2 = post_data.groupby(post_data.time.dt.hour)
total_posts_n = len(post_data)
gr2 = dict(gr2.count().time)
index = range(24)
values = [(gr2[i] / total_posts_n) * 100 if i in gr2 else 0 for i in index]

radar_chart = pygal.Radar(show_legend=False, fill=True, height=800)
radar_chart.x_labels = index
radar_chart.add('', values)



In [ ]:
# Wiadomości z obrazkami
msg = data['messages']
# msg[msg.conversation == 'Andrzej Tat'].photos.dtype
# msg[msg.conversation == '8 Ball Pool Game']
photo_count = msg.groupby(msg.conversation).photos.sum().sort_values()
photo_count = photo_count[photo_count != 0]
chart = pygal.HorizontalBar(show_legend=False)
chart.x_labels = list(photo_count.index)
chart.add('', photo_count)

In [ ]:
# change:
# * Posts graphs
# ! change to % in hour graph
# ! Rozmiar konwersacji w Bajtach
# ! clear all outputs, przed commitem
# ! mój plik z danymi nie działa

In [ ]:
t = [0, 0, 0, 0, 0, 0, 1]
d = [0, 0, 0, 0, 0, 0, 0]

df = pd.DataFrame({'name': ['jacek'] * len(t) + ['andrzej'] * len(d), 'photo': t + d})

df.groupby(df.name).photo.sum()
